In [1]:
import pandas as pd
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import RobustScaler, LabelEncoder
import joblib

entorno = 'local'  # Elegir "VM" o "local" para correr en entorno local
correr_productos_mayor_ventas = True #Seleccionar True para correr solo en los productos de mas ventas del listado lista_productos_mayor_ventas
nombre_experimento = 'LSTM_producto_cliente'
nombre_archivo_log = 'log_predicciones.csv'
ventana_input = 12
ventana_output = 2
ventana_test = 3
lags = 6

lista_productos_mayor_ventas = [20001 ,20002 ,20003 ,20004 ,20005 ,20032 ,20009 ,20006 ,20011 ,20007 ,20010 ,20019 ,20013 ,20015 ,20016 ,20014 ,20024 ,20020 ,20025 ,20026]

# Configurar entorno
if entorno == 'VM':
    carpeta_datasets = '~/buckets/b1/datasets'
    carpeta_exp_base = '~/buckets/b1/exp'
elif entorno == 'local':
    carpeta_datasets = 'C:\\Users\\alope\\Desktop\\Trámites\\Maestria Data Science - Universidad Austral\\Laboratorio de implementación 3\\Datos'
    carpeta_exp_base = 'C:\\Users\\alope\\Desktop\\Trámites\\Maestria Data Science - Universidad Austral\\Laboratorio de implementación 3\\Resultados'
else:
    raise Exception("Entorno especificado incorrectamente")

carpeta_exp = os.path.join(carpeta_exp_base, nombre_experimento)
if not os.path.exists(carpeta_exp):
    os.makedirs(carpeta_exp)
    
dataset_completo = pd.read_csv(os.path.join(carpeta_datasets, 'dataset_completo.csv'))

if correr_productos_mayor_ventas:
    dataset_completo = dataset_completo[dataset_completo['product_id'].isin(lista_productos_mayor_ventas)]

#Eliminar columnas no utilizadas
dataset_completo.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,...,sku_size,descripcion,mes,quarter,fin_quarter,edad_producto,ventas_cat1,ventas_cat2,ventas_cat3,ventas_familia_producto
4145,201701,10063,20007,0,1,0.81730,0.81730,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,...,900.0,Sabor 13,1,1,0,0,16.71557,3.26920,3.26920,0.81730
4146,201701,10046,20007,0,1,8.17296,8.17296,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,...,900.0,Sabor 13,1,1,0,0,53.07292,10.12942,9.17086,8.17296
4147,201701,10027,20007,0,2,0.38919,0.38919,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,...,900.0,Sabor 13,1,1,0,0,40.40341,7.21172,5.35810,2.55135
4148,201701,10045,20007,0,1,1.63460,1.63460,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,...,900.0,Sabor 13,1,1,0,0,99.03031,11.39925,9.79178,4.01298
4149,201701,10488,20007,0,1,0.24324,0.24324,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,...,900.0,Sabor 13,1,1,0,0,2.47952,0.24324,0.24324,0.24324


In [2]:
#Chequear si existe log de productos con su prediccion
if os.path.exists(os.path.join(carpeta_exp, nombre_archivo_log)):
    # Si existe archivo, leerlo
    productos_con_prediccion = pd.read_csv(os.path.join(carpeta_exp,nombre_archivo_log))
else:
    #Si el archivo no existe
    productos_con_prediccion = pd.DataFrame(columns = ['product_id' , 'customer_id', 'prediccion'])


# Crear una columna combinada para poder filtrar las combinaciones product/cliente ya predichas
productos_con_prediccion['product_customer_id'] = productos_con_prediccion['product_id'].astype(str) + '_' + productos_con_prediccion['customer_id'].astype(str)
dataset_completo['product_customer_id'] = dataset_completo['product_id'].astype(str) + '_' + dataset_completo['customer_id'].astype(str)

# Filtrar por la dupla customer_id y product_id
df = dataset_completo[~dataset_completo['product_customer_id'].isin(productos_con_prediccion['product_customer_id'].values)]

# Eliminar la columna combinada si ya no es necesaria
df = df.drop(columns=['product_customer_id'])
dataset_completo = dataset_completo.drop(columns=['product_customer_id'])
productos_con_prediccion = productos_con_prediccion.drop(columns=['product_customer_id'])

df

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,...,sku_size,descripcion,mes,quarter,fin_quarter,edad_producto,ventas_cat1,ventas_cat2,ventas_cat3,ventas_familia_producto
10753,201701,10395,20005,0,1,0.08590,0.08590,FOODS,ADEREZOS,Mayonesa,...,120.0,Regular sin TACC,1,1,0,0,0.27734,0.24642,0.24642,0.24642
10754,201701,10195,20005,0,1,2.06170,2.06170,FOODS,ADEREZOS,Mayonesa,...,120.0,Regular sin TACC,1,1,0,0,8.22598,6.79992,5.12361,4.95255
10755,201701,10225,20005,0,1,0.42952,0.42952,FOODS,ADEREZOS,Mayonesa,...,120.0,Regular sin TACC,1,1,0,0,3.18760,2.16183,1.87006,1.76978
10756,201701,10288,20005,0,1,0.17181,0.17181,FOODS,ADEREZOS,Mayonesa,...,120.0,Regular sin TACC,1,1,0,0,0.91607,0.60803,0.52716,0.17181
10757,201701,10158,20005,0,1,0.04295,0.04295,FOODS,ADEREZOS,Mayonesa,...,120.0,Regular sin TACC,1,1,0,0,6.54173,4.12110,3.11612,1.27626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2292109,201912,10159,20014,0,2,0.16380,0.16380,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,...,3000.0,Sabor 13,12,4,1,35,2.56273,0.57369,0.41884,0.18326
2292110,201912,10291,20014,0,1,0.16380,0.16380,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,...,3000.0,Sabor 13,12,4,1,35,5.68662,0.81823,0.59983,0.45569
2292111,201912,10022,20014,0,1,4.35708,4.35708,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,...,3000.0,Sabor 13,12,4,1,35,339.13673,19.79160,18.85652,9.26086
2292112,201912,10013,20014,0,3,18.73872,18.73872,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,...,3000.0,Sabor 13,12,4,1,35,475.42938,38.30299,34.55525,18.73872


In [3]:
#Convertir periodo en index
df['Timestamp'] = pd.to_datetime(df['periodo'], format='%Y%m')
df.set_index('Timestamp', inplace=True)
df.drop(columns=['periodo'], inplace=True)
df.head()


,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion,mes,quarter,fin_quarter,edad_producto,ventas_cat1,ventas_cat2,ventas_cat3,ventas_familia_producto
Timestamp,,,,,,,,,,,,,,,,,,,,
2017-01-01,10395,20005,0,1,0.08590,0.08590,FOODS,ADEREZOS,Mayonesa,NATURA,120.0,Regular sin TACC,1,1,0,0,0.27734,0.24642,0.24642,0.24642
2017-01-01,10195,20005,0,1,2.06170,2.06170,FOODS,ADEREZOS,Mayonesa,NATURA,120.0,Regular sin TACC,1,1,0,0,8.22598,6.79992,5.12361,4.95255
2017-01-01,10225,20005,0,1,0.42952,0.42952,FOODS,ADEREZOS,Mayonesa,NATURA,120.0,Regular sin TACC,1,1,0,0,3.18760,2.16183,1.87006,1.76978
2017-01-01,10288,20005,0,1,0.17181,0.17181,FOODS,ADEREZOS,Mayonesa,NATURA,120.0,Regular sin TACC,1,1,0,0,0.91607,0.60803,0.52716,0.17181
2017-01-01,10158,20005,0,1,0.04295,0.04295,FOODS,ADEREZOS,Mayonesa,NATURA,120.0,Regular sin TACC,1,1,0,0,6.54173,4.12110,3.11612,1.27626


In [ ]:
#Completar dataframe por series productos-clientes en meses vacios
#Cargar los valores de ventas agrupadas por periodo
ventas_cat1 = joblib.load(os.path.join(carpeta_datasets,'ventas_cat1.pkl'))
ventas_cat2 = joblib.load(os.path.join(carpeta_datasets,'ventas_cat2.pkl'))
ventas_cat3 = joblib.load(os.path.join(carpeta_datasets,'ventas_cat3.pkl'))
ventas_familia_productos = joblib.load(os.path.join(carpeta_datasets,'ventas_familia_productos.pkl'))
#Convertir los periodos a timestamp
ventas_cat1['Timestamp'] = pd.to_datetime(ventas_cat1['periodo'], format='%Y%m')
ventas_cat2['Timestamp'] = pd.to_datetime(ventas_cat2['periodo'], format='%Y%m')
ventas_cat3['Timestamp'] = pd.to_datetime(ventas_cat3['periodo'], format='%Y%m')
ventas_familia_productos['Timestamp'] = pd.to_datetime(ventas_familia_productos['periodo'], format='%Y%m')
ventas_cat1.drop(columns=['periodo'], inplace=True)
ventas_cat2.drop(columns=['periodo'], inplace=True)
ventas_cat3.drop(columns=['periodo'], inplace=True)
ventas_familia_productos.drop(columns=['periodo'], inplace=True)


# Fecha final del dataset completo
fecha_final = df.index.max()

df_producto_cliente_completo = pd.DataFrame()

if os.path.exists(os.path.join(carpeta_exp, 'df_producto_cliente_completo.csv')):
    df_producto_cliente_completo = pd.read_csv(os.path.join(carpeta_exp, 'df_producto_cliente_completo.csv'))
else:
    for producto in df['product_id'].unique():
        ventas_mes_por_producto = df[df['product_id'] == producto].copy()
        for cliente in ventas_mes_por_producto['customer_id'].unique():
            ventas_mes_por_producto_cliente = ventas_mes_por_producto[ventas_mes_por_producto['customer_id'] == cliente].copy()
            fecha_inicial = ventas_mes_por_producto_cliente.index.min()
            date_range = pd.date_range(start=fecha_inicial, end=fecha_final, freq='MS')
            ventas_mes_por_producto_cliente = ventas_mes_por_producto_cliente.reindex(date_range)
            ventas_mes_por_producto_cliente = ventas_mes_por_producto_cliente.reset_index().rename(columns= {'index': 'Timestamp'}) #Volvemos la columna Timestamp para poder hacer merge
            #Rellenar los valores faltantes generados
            ventas_mes_por_producto_cliente['tn'] = ventas_mes_por_producto_cliente['tn'].fillna(0)
            ventas_mes_por_producto_cliente['cust_request_qty'] = ventas_mes_por_producto_cliente['cust_request_qty'].fillna(0)
            ventas_mes_por_producto_cliente['cust_request_tn'] = ventas_mes_por_producto_cliente['cust_request_tn'].fillna(0)
            ventas_mes_por_producto_cliente['product_id'] = producto
            ventas_mes_por_producto_cliente['customer_id'] = cliente
            for index,fila in ventas_mes_por_producto_cliente.iterrows():
                if pd.isna(fila['plan_precios_cuidados']):
                    ventas_mes_por_producto_cliente.loc[index, 'plan_precios_cuidados'] = df[(df['product_id'] == producto) & (df.index == fila['Timestamp'])]['plan_precios_cuidados'].iloc[0]
                    ventas_mes_por_producto_cliente.loc[index, 'cat1'] = df[df['product_id'] == producto]['cat1'].iloc[0]
                    ventas_mes_por_producto_cliente.loc[index, 'cat2'] = df[df['product_id'] == producto]['cat2'].iloc[0]
                    ventas_mes_por_producto_cliente.loc[index, 'cat3'] = df[df['product_id'] == producto]['cat3'].iloc[0]
                    ventas_mes_por_producto_cliente.loc[index, 'brand'] = df[df['product_id'] == producto]['brand'].iloc[0]
                    ventas_mes_por_producto_cliente.loc[index, 'descripcion'] = df[df['product_id'] == producto]['descripcion'].iloc[0]
                    ventas_mes_por_producto_cliente.loc[index, 'sku_size'] = df[df['product_id'] == producto]['sku_size'].iloc[0]
                    ventas_mes_por_producto_cliente.loc[index, 'mes'] = df[(df['product_id'] == producto) & (df.index == fila['Timestamp'])]['mes'].iloc[0]
                    ventas_mes_por_producto_cliente.loc[index, 'quarter'] = df[(df['product_id'] == producto) & (df.index == fila['Timestamp'])]['quarter'].iloc[0]
                    ventas_mes_por_producto_cliente.loc[index, 'fin_quarter'] = df[(df['product_id'] == producto) & (df.index == fila['Timestamp'])]['fin_quarter'].iloc[0]
                    ventas_mes_por_producto_cliente.loc[index, 'edad_producto'] = df[(df['product_id'] == producto) & (df.index == fila['Timestamp'])]['edad_producto'].iloc[0]
                    ventas_mes_por_producto_cliente.loc[index, 'ventas_cat1'] = df[(df['product_id'] == producto) & (df.index == fila['Timestamp'])]['ventas_cat1'].iloc[0]
                    ventas_mes_por_producto_cliente.loc[index, 'ventas_cat2'] = df[(df['product_id'] == producto) & (df.index == fila['Timestamp'])]['ventas_cat2'].iloc[0]
                    ventas_mes_por_producto_cliente.loc[index, 'ventas_cat3'] = df[(df['product_id'] == producto) & (df.index == fila['Timestamp'])]['ventas_cat3'].iloc[0]
                    ventas_mes_por_producto_cliente.loc[index, 'ventas_familia_producto'] = df[(df['product_id'] == producto) & (df.index == fila['Timestamp'])]['ventas_familia_producto'].iloc[0]
            df_producto_cliente_completo = pd.concat([df_producto_cliente_completo, ventas_mes_por_producto_cliente], axis = 0)

    df_producto_cliente_completo.to_csv(os.path.join(carpeta_exp, 'df_producto_cliente_completo.csv'), index = False)

In [ ]:
#Encoding de columnas categoricas
''''
columnas_categoricas = ['cat1', 'cat2', 'cat3', 'brand']

dict_encoders = {}
for columna in columnas_categoricas:
    encoder = LabelEncoder()
    df[columna] = encoder.fit_transform(df[columna])
    dict_encoders[columna] = encoder

joblib.dump(dict_encoders, os.path.join(carpeta_exp,'label_encoder.pkl')) #Guardar el label encoder en un pickle

'''
###Por ahora eliminamos cat1, cat2, cat3, porque son iguales en todas las filas
df_producto_cliente_completo.drop(columns=['sku_size', 'descripcion', 'cust_request_qty', 'cust_request_tn','cat1', 'cat2', 'cat3', 'brand'], inplace = True)
df_producto_cliente_completo.set_index('Timestamp', inplace = True)

In [ ]:
# Crear lags y rellenar NA
def crear_lags(df, lags):
    for lag in range(1, lags + 1):
        df[f'tn_lag_{lag}'] = df['tn'].shift(lag)
        df[f'tn_lag_{lag}'].bfill(inplace=True) # Usar backfill para llenar los valores NaN
        df[f'ventas_cat1_lag_{lag}'] = df['ventas_cat1'].shift(lag)
        df[f'ventas_cat1_lag_{lag}'].bfill(inplace=True) # Usar backfill para llenar los valores NaN
        df[f'ventas_cat2_lag_{lag}'] = df['ventas_cat2'].shift(lag)
        df[f'ventas_cat2_lag_{lag}'].bfill(inplace=True) # Usar backfill para llenar los valores NaN
        df[f'ventas_cat3_lag_{lag}'] = df['ventas_cat3'].shift(lag)
        df[f'ventas_cat3_lag_{lag}'].bfill(inplace=True) # Usar backfill para llenar los valores NaN
        df[f'ventas_familia_producto_lag_{lag}'] = df['ventas_familia_producto'].shift(lag)
        df[f'ventas_familia_producto_lag_{lag}'].bfill(inplace=True) # Usar backfill para llenar los valores NaN
    return df

def crear_dataset_supervisado(array, input_length, output_length):
    # Inicialización
    X, Y = [], []    # Listados que contendrán los datos de entrada y salida del modelo
    shape = array.shape
    if len(shape)==1: # Si tenemos sólo una serie (univariado)
        fils, cols = array.shape[0], 1
        array = array.reshape(fils,cols)
    else: # Multivariado
        fils, cols = array.shape
    # Generar los arreglos (utilizando ventanas deslizantes de longitud input_length)
    for i in range(fils-input_length-output_length + 1):
        X.append(array[i:i+input_length,0:cols])
        Y.append(array[i+input_length:i+input_length+output_length,-1].reshape(output_length,1))

    # Convertir listas a arreglos de NumPy
    X = np.array(X)
    Y = np.array(Y)

    return X, Y

# Definir función para crear y entrenar el modelo
def crear_modelo(ventana_input, ventana_output, lstm_units=[64, 32], dropout_rate=0.5, recurrent_dropout=0.25):
    model = Sequential()
    model.add(LSTM(lstm_units[0], return_sequences=True, input_shape=(ventana_input, X.shape[2]), recurrent_dropout=recurrent_dropout))
    for units in lstm_units[1:]:
        model.add(LSTM(units, recurrent_dropout=recurrent_dropout))
    model.add(Dropout(dropout_rate))
    model.add(Dense(ventana_output))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
#Cargar scalers si ya existen creados
if os.path.exists(os.path.join(carpeta_exp, 'scalers.pkl')):
    # Si existe archivo, leerlo
    scalers_dict = joblib.load(os.path.join(carpeta_exp, 'scalers.pkl'))
else:
    scalers_dict = {}


EPOCHS = 100
BATCH_SIZE = 64

#Cantidad de productos pendientes de predecir
cant_productos_pendientes = len(df['product_id'].unique())

# Entrenar y predecir
i = 0
for producto in df_producto_cliente_completo['product_id'].unique():
    i += 1
    ventas_mes_por_producto = df_producto_cliente_completo[df_producto_cliente_completo['product_id'] == producto].copy()
    for cliente in ventas_mes_por_producto['customer_id'].unique():
        print(f'Entrenando cliente: {cliente}, producto: {producto}')
        ventas_mes_por_producto_cliente = ventas_mes_por_producto[ventas_mes_por_producto['customer_id'] == cliente].copy()
        ventas_mes_por_producto_cliente.drop(columns=['product_id', 'customer_id'], inplace=True)
        

        lista_scalers = []
        for columna in ['tn', 'edad_producto', 'ventas_cat1', 'ventas_cat2', 'ventas_cat3', 'ventas_familia_producto']:
            scaler = RobustScaler()
            ventas_mes_por_producto_cliente[columna] = scaler.fit_transform(ventas_mes_por_producto_cliente[columna].values.reshape(-1,1))
            lista_scalers.append(scaler)
        scalers_dict[f'{producto}-{cliente}'] = lista_scalers

        ventas_mes_por_producto_cliente = crear_lags(ventas_mes_por_producto_cliente, lags)
        X, Y = crear_dataset_supervisado(np.array(ventas_mes_por_producto_cliente), ventana_input, ventana_output)

        
        model = crear_modelo(ventana_input, ventana_output)
        model.fit(X, Y, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1)
        y_pred_s = model.predict(X[-1].reshape(1,X.shape[1],X.shape[2]), verbose=0)
        y_pred = scaler.inverse_transform(y_pred_s)
        prediccion_mes_2 = y_pred.flatten()[1]
        nueva_prediccion = pd.DataFrame({'product_id': [producto], 'customer_id': [cliente], 'prediccion': [prediccion_mes_2]})
        productos_con_prediccion = pd.concat([productos_con_prediccion, nueva_prediccion], axis = 0)
        productos_con_prediccion.to_csv(os.path.join(carpeta_exp,nombre_archivo_log), index = False)